In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4786e7d45c1e2a8da5d2e7cc70c3d48a0dbf560fd80bacd80d6cad7428c752cb
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 52.3 MB/s eta 0:00:00


In [ ]:
from langchain.llms import GooglePalm
from langchain.vectorstores import FAISS
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

import pandas as pd

In [ ]:
#https://data.world/opensnippets/ebay-uk-products-dataset
products = pd.read_csv('https://query.data.world/s/q533sypivreoqflj4asecuqpytcif4?dws=00000', usecols=['name'])

In [ ]:
products.head()

,name
0,2x Ultra-thin LCD Digital Display Vehicle Car ...
1,MERCEDES VANEO 414 1.6 Pollen / Cabin Filter 0...
2,Windscreen Kappa Transparent 58x45cm for April...
3,HP LaserJet Enterprise M552DN Colour 1200 X 12...
4,Honda RS 250 R 1991-1994 DP Brakes Street Sint...


In [ ]:
# Modify the llmChain prompt template for generating a list of 10 items
prompt_template = '''<s>[INST] <<SYS>>
Only tell me the product names. The answer should only include ten names.
<</SYS>>

{context}[/INST]'''

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                   model_kwargs={'device': 'cpu'})

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
product_names = products['name'].values.astype(str)
product_embeddings = FAISS.from_texts(product_names, embeddings)

In [ ]:
from langchain.llms import GooglePalm

api_key = '*****************' # free api key from https://makersuite.google.com/

palm_llm = GooglePalm(google_api_key=api_key, temperature=0.1)

In [ ]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context"])
llm_chain = LLMChain(prompt=PROMPT, llm=palm_llm)
d
# Run Semantic Search using llmChain
raw_query = 'What are the best gifts for boys under 5?'
enhanced_query = llm_chain.run({'context': raw_query})
print(enhanced_query)

1. **LEGO DUPLO Brick Box**
2. **Fisher-Price Laugh & Learn Smart Stages Puppy**
3. **VTech Switch & Go Dinos Rex**
4. **Melissa & Doug Water Wow! Animals**
5. **Magformers Magnetic Building Set**
6. **B. toys Activity Cube**
7. **Hape Rainbow Stacker**
8. **Play-Doh Creative Playset**
9. **Squishmallows Plush Toy**
10. **Little Tikes Cozy Coupe**


In [ ]:
product_embeddings.similarity_search_with_score(enhanced_query, k=10)

[(Document(page_content='Count The Toys a Fun Picture Adding up Book for 2-5 Year Olds (pb) 1976832578'),
  0.7624263),
 (Document(page_content='Cutetitos Fruititos 39144 Surprise Stuffed Animals Cute Plush Surprise Toys 2'),
  0.8314363),
 (Document(page_content='Dqtye 6pcs Bath Toy Floating Sea Animal Baby Soft Squirt Bathing Toys Bathtub'),
  0.92065203),
 (Document(page_content='Lovable Amigurumi Toys: 15 Doll Crochet Projects by Lilleliis by Mari-Liis Lille (Paperback, 2019)'),
  0.9693754),
 (Document(page_content='Crayola Shimmer Activity Pack 3 Books and 90 Stickers'),
  0.9987101),
 (Document(page_content='LEGO 76395 Harry Potter Hogwarts First Flying Lesson Age 7 264pcs'),
  1.0115043),
 (Document(page_content='Intex Underwater Fish Dive Rings Kids Swimming Pool Diving Toy Play Sticks 4 Set'),
  1.012012),
 (Document(page_content='LEGO Dimensions Powerpuff Girls Fun Pack 71343'),
  1.0134681),
 (Document(page_content='13x Halloween Cauldrons Witch Buckets Kids Trick or Treat 